In [11]:
import os

In [12]:
%pwd

'c:\\Users\\umair\\Desktop\\ML\\waterQualityPrediction'

In [2]:
os.chdir("../")

In [34]:
from dataclasses import dataclass
from pathlib import Path

In [55]:
@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    max_depth: int
    n_estimators: int
    n_jobs: int
    

In [56]:
from waterQualityPrediction.constants import *
from waterQualityPrediction.utils import read_yaml, create_directories

In [57]:
from sklearn.ensemble import RandomForestClassifier

In [58]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainingConfig:
        config = self.config.model_trainer
        params = self.params.RandomForest

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainingConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            max_depth = params.max_depth,
            n_estimators = params.n_estimators,
            n_jobs = params.n_jobs
            
        )

        return model_trainer_config


In [59]:
import pandas as pd
import os
from waterQualityPrediction import logger
import joblib

In [69]:
class ModelTrainer:
    def __init__(self, config: ModelTrainingConfig, params: ModelTrainingConfig):
        self.config = config
        self.params = params
   
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
                    
        train_x = train_data.iloc[:, :-1]
        train_y = train_data.iloc[:, -1]
        test_x = test_data.iloc[:, :-1]
        test_y = test_data.iloc[:, -1]   

        # train_x = train_data.drop([self.config.target_column], axis=1)
        # test_x = test_data.drop([self.config.target_column], axis=1)
        # train_y = train_data[[self.config.target_column]]
        # test_y = test_data[[self.config.target_column]]


        random_forest = RandomForestClassifier(max_depth=self.params.max_depth,
                                               n_estimators=self.params.n_estimators, 
                                               n_jobs=self.params.n_jobs)
        random_forest.fit(train_x, train_y)

        joblib.dump(random_forest, os.path.join(self.config.root_dir, self.config.model_name))

In [73]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config, params=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e


[2024-01-26 17:10:50,631: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-26 17:10:50,631: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-26 17:10:50,640: INFO: common: created directory at: artifacts]
[2024-01-26 17:10:50,643: INFO: common: created directory at: artifacts/model_trainer]


c:\Users\umair\miniconda3\envs\water\lib\site-packages\pandas\core\generic.py:2150: RuntimeWarning: overflow encountered in cast
  arr = np.asarray(values, dtype=dtype)


ValueError: Input X contains infinity or a value too large for dtype('float32').